In [ ]:
num = 2

In [ ]:
!pip install -U FlagEmbedding
!pip install einops
!pip install gdown

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

In [ ]:
import pandas as pd

data = pd.read_csv('/kaggle/input/test-dataa/test_data.csv') 
questions = data['question'].tolist()
qids = data['qid'].tolist()


In [ ]:
import json
with open('/kaggle/input/template/template_chunking_flat.json', 'r', encoding='utf-8') as f:
    text_data = json.load(f)
    
text_dict = {str(item['stt']): item['text'] for item in text_data}
stt_dict = {item['text']: item['stt'] for item in text_data}


In [ ]:
qid_id_data = []
with open(f'/kaggle/input/data-phase-1/test/bge+e5/output_part_{num}.txt', 'r') as file:
    for line in file:
        qid_and_ids = line.strip().split()
        qid = qid_and_ids[0]
        ids = qid_and_ids[1:]

        question = data[data['qid'] == int(qid)]['question'].values[0]

        answers = [text_dict.get(id, '') for id in ids] 
        qid_id_data.append((qid, question, answers))


In [ ]:
from tqdm import tqdm

rerank_results = []
scores_results = []
for qid, question, answers in tqdm(qid_id_data, desc="Processing QID data"):
    question_answer_pairs = [(question, answer) for answer in answers if answer]  

    if not question_answer_pairs:
        continue

    pairs = [(question, answer) for question, answer in question_answer_pairs]
    scores = reranker.compute_score(pairs, normalize=True)

    ranked_answers = [(answer, score) for (answer, score) in zip(answers, scores)]
    ranked_answers.sort(key=lambda x: x[1], reverse=True) 

    ranked_stts_scores = [(stt_dict[answer], score) for (answer, score) in ranked_answers]
    rerank_results.append((qid, [stt for stt, _ in ranked_stts_scores]))  
    scores_results.append((qid, [score for _, score in ranked_stts_scores]))  

with open(f'reranked_output{num}.txt', 'w') as f:
    for qid, ranked_stts in tqdm(rerank_results, desc="Writing STT results to file"):
        result_line = f"{qid} " + " ".join(map(str, ranked_stts))
        f.write(result_line + "\n")

with open(f'reranked_scores{num}.txt', 'w') as f:
    for qid, scores in tqdm(scores_results, desc="Writing Scores to file"):
        result_line = f"{qid} " + " ".join(f"{score:.4f}" for score in scores)
        f.write(result_line + "\n")


In [ ]:
with open('/kaggle/input/template/template_chunking_flat.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

stt_to_infor_id = {str(item['stt']): int(item['infor_id']) for item in json_data}

reranked_results = []
with open(f'reranked_scores{num}.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        if not parts:
            continue
        qid = parts[0]
        ids = parts[1:]
        top_10_ids = ids[:10]
        infor_ids = [stt_to_infor_id.get(i, i) for i in top_10_ids]
        reranked_results.append(f"{qid} " + " ".join(map(str, infor_ids)))

with open(f'reranked_infor_output_part{num}.txt', 'w', encoding='utf-8') as f:
    for result in reranked_results:
        f.write(result + "\n")

In [ ]:
import zipfile

input_file = f'reranked_infor_output_part{num}.txt'  
output_zip_file = f'reranked_infor_output_part{num}.zip'  

with zipfile.ZipFile(output_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(input_file, arcname=input_file.split('/')[-1])  

print(f"Đã nén file {input_file} thành {output_zip_file}")
